In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fbprophet import Prophet
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import acquire
import prepare

In [ ]:
df = acquire.get_all_data()

In [ ]:
df.head()

### Looking only at store 1

In [ ]:
d_df = df[df['store_id']==1]


### Looking only at the Kelloggs items that were sold

In [ ]:
d_df = d_df[d_df['item_brand']=='Kelloggs']

In [ ]:
d_df.head()

In [ ]:
d_df.info()

In [ ]:
d_df['y'] = d_df.sale_amount * d_df.item_price

In [ ]:
d_df.head()

In [ ]:
d_df['ds'] = pd.to_datetime(d_df.sale_date)

In [ ]:
d_df.head()

In [ ]:
d_df = d_df.groupby(['ds'])['y'].sum().reset_index()


In [ ]:
d_df.head()

In [ ]:
plt.figure(figsize=(16,6))
sns.tsplot(d_df.y)

In [ ]:
d_df['cap'] = 600
d_df['floor'] = 100

m = Prophet(daily_seasonality=True, growth='logistic', changepoint_range=0.9)
m.fit(d_df)

In [ ]:
future = m.make_future_dataframe(periods=180)
future['cap'] = 600
future['floor'] = 100
print(future.head())
print(future.tail())
print(d_df.tail())

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)


In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
from fbprophet.diagnostics import cross_validation, performance_metrics
# cross_validation(m, initial = 730, period = 180, horizon = 365, units = 'days')
df_cv = cross_validation(m, horizon='90 days')


df_p = performance_metrics(df_cv)
df_p.head(5)

In [ ]:
from fbprophet.plot import plot_cross_validation_metric, plot_forecast_component

# plot_forecast_component(m, forecast, 'yhat')

fig3 = plot_cross_validation_metric(df_cv, metric='rmse')